In [15]:
#import dependencies
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import pandas as pd
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import func


In [4]:
# Step 1: Read CSV Files into DataFrames
industry_groups_df = pd.read_csv('Resources/Industry_Groups.csv')
top_ten_df = pd.read_csv('Resources/top_ten_with_industryID.csv')
top_ten_historic_df = pd.read_csv('Resources/top_ten_historic_5y.csv')
fundamentals_df = pd.read_csv('Resources/02_ASX_Fundamental_Final_Clean.csv')
fundamentals_df.head()

,Ticker,lastPrice,Change,Bid_Ask,volumePerDay,volume4wAvg,Open,dayRange,prevClose,lastTrade,...,sharesIssued,Sector,similarCompanies,EPS,DPS,bookValuePerShare,Breakdown,Recommendation,lastUpdated,PE
0,ARB,31.26,1.02 (3.37%),$29.88 - $31.35,160245.0,245483.0,31.07,$30.56 - $31.45,$30.24,6:58pm 03/11 (AEDT),...,82220441.0,Consumer Cyclical,CTD / WEB / SUL,1.079,0.6200,0.600,6 Buy · 5 Hold · 2 Sell,Hold,01/10/23,28.971270
1,GUD,11.08,0.17 (1.56%),$10.77 - $11.35,366353.0,441797.0,10.95,$10.725 - $11.10,$10.91,6:58pm 03/11 (AEDT),...,140894696.0,Consumer Cyclical,SGR / LNW / SKC,0.693,0.3900,0.526,11 Buy · 2 Hold · 0 Sell,Strong Buy,01/10/23,15.988456
2,PWH,10.12,0.07 (0.70%),$9.96 - $10.92,37398.0,127350.0,10.16,$10.06 - $10.22,$10.05,6:58pm 03/11 (AEDT),...,100484131.0,Consumer Cyclical,BGP / CTT / AX1,0.217,0.1250,0.722,7 Buy · 4 Hold · 1 Sell,Buy,01/10/23,46.635945
3,SFC,16.10,0.65 (-3.88%),$16.15 - $17.10,5570.0,758.0,16.75,$16.01 - $16.75,$16.75,6:58pm 03/11 (AEDT),...,13569557.0,Consumer Cyclical,NDO / ALG / BST,1.811,0.9000,3.324,NaN,NaN,NaN,8.890116
4,AHL,0.89,0.00 (0.00),$0.845 - $0.90,0.0,11146.0,NaN,‐ - ‐,$0.89,6:58pm 03/11 (AEDT),...,81285172.0,Consumer Cyclical,ISU / KED / CCX,0.084,0.0233,0.986,NaN,NaN,NaN,10.595238


In [ ]:
# #trying to see if i had any null values in 'Date' column of top_ten_historic_df
# print(top_ten_historic_df['Ticker'].isnull().sum())

In [6]:
top_ten_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ticker        259 non-null    object 
 1   company_name  259 non-null    object 
 2   market_cap    259 non-null    float64
 3   industry_id   259 non-null    int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 8.2+ KB


In [7]:
#convert datatype to date
top_ten_historic_df['Date'] = pd.to_datetime(top_ten_historic_df['Date'])


In [8]:
top_ten_historic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60881 entries, 0 to 60880
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Ticker     60881 non-null  object        
 1   Date       60881 non-null  datetime64[ns]
 2   Open       60881 non-null  float64       
 3   High       60881 non-null  float64       
 4   Low        60881 non-null  float64       
 5   Close      60881 non-null  float64       
 6   Adj Close  60881 non-null  float64       
 7   Volume     60881 non-null  int64         
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 3.7+ MB


In [9]:
industry_groups_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   industry_name  27 non-null     object
 1   industry_id    27 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 560.0+ bytes


In [17]:
# Define Database Schema
Base = declarative_base()

class IndustryGroups(Base):
    __tablename__ = 'industry_groups'
    
    industry_name = Column(String)
    industry_id = Column(Integer, primary_key=True)
    

class TopTen(Base):
    __tablename__ = 'top_ten'
    
    ticker = Column(String, primary_key=True)
    company_name = Column(String)
    market_cap = Column(Float)
    industry_id = Column(Integer, ForeignKey('industry_groups.industry_id'))

class TopTenHistoric(Base):
    __tablename__ = 'top_ten_historic'
    
    id = Column (Integer, primary_key=True, autoincrement=True)
    Ticker = Column(String, ForeignKey('top_ten.ticker'))
    Date = Column(Date)
    Open = Column(Float)
    High = Column(Float)
    Low = Column(Float)
    Close = Column(Float)
    Adj_Close = Column('Adj Close', Float)
    Volume = Column(Integer)

class Fundamentals(Base):
    __tablename__ = 'fundamentals'
    
    fundamental_ID = Column(Integer, primary_key=True)
    Ticker = Column(String, ForeignKey('top_ten.ticker'))
    lastPrice = Column(Float)
    Change = Column(String) 
    Bid_Ask = Column(String) 
    volumePerDay = Column(Float)
    volume4wAvg= Column(Float)
    Open = Column(Float)
    dayRange = Column(String)
    prevClose = Column(String) 
    lastTrade = Column(String)
    oneWeek = Column(String)
    oneMonth = Column(String)
    YTD2023 = Column(String)
    oneYear = Column(String)
    vsSectorOneYr = Column(String)
    vsASX200OneYr = Column(String)
    marketCap = Column(Float)
    ASXRank = Column(String)
    sectorRank = Column(String)
    sharesIssued = Column(Float)
    Sector = Column(String)
    similarCompanies = Column(String)
    EPS = Column(Float)
    DPS = Column(Float)
    bookValuePerShare = Column(Float)
    Breakdown = Column(String)
    Recommendation = Column(String)
    lastUpdated = Column(String)
    PE = Column(Float)



In [18]:
#Connect to the Database and Create Tables
# Replace 'sqlite:///asx.db' with your actual database connection string
engine = create_engine('sqlite:///Resources/top_ten_asx.db')
Base.metadata.create_all(engine)

In [19]:
#Insert DataFrames into Database
Session = sessionmaker(bind=engine)
session = Session()

# Insert IndustryGroups
session.bulk_insert_mappings(IndustryGroups, industry_groups_df.to_dict(orient="records"))

# Insert TopTen
session.bulk_insert_mappings(TopTen, top_ten_df.to_dict(orient="records"))

# Insert TopTenHistoric
session.bulk_insert_mappings(TopTenHistoric, top_ten_historic_df.to_dict(orient="records"))

# Insert Fundamentals
session.bulk_insert_mappings(Fundamentals, fundamentals_df.to_dict(orient="records"))



In [20]:
#commit changes
session.commit()

In [21]:
#import dependencies for inspector
from sqlalchemy import inspect

In [22]:
# Instantiate an Inspector with the engine
inspector = inspect(engine)

In [23]:
# Get a list of table names
table_names = inspector.get_table_names()
print(table_names)

['fundamentals', 'industry_groups', 'top_ten', 'top_ten_historic']


In [24]:
#Get columns of top_ten_historic
columns = inspector.get_columns('top_ten')
for column in columns:
    print(column['name'])

ticker
company_name
market_cap
industry_id


In [25]:
#reflect database into ORM classes 
Base = automap_base()
Base.prepare(autoload_with=engine)
Base.classes.keys()

['fundamentals', 'top_ten', 'industry_groups', 'top_ten_historic']

In [26]:
#save a reference to the top_ten table as 'top_ten'
top_ten = Base.classes.top_ten

In [29]:
from sqlalchemy.orm import Session

In [30]:
#create a database session object
session = Session(engine)

In [32]:
# #list all of the tickers found in the top_ten table 
# session.query(top_ten.ticker).all()

## Querying Existing tables

In [33]:
 # Create engine using the `top_ten_asx.db` database file
engine = create_engine('sqlite:///Resources/top_ten_asx.db')


In [34]:
#Declare a Base using automap_base()
Base = automap_base()

In [35]:
# Use the Base class to reflect the database tables
#examines the database, discover the tables and columns and relationships within the db, and set up Python classes.
Base.prepare(autoload_with=engine)

In [36]:
# Print all of the classes mapped to the Base
Base.classes.keys()

['fundamentals', 'top_ten', 'industry_groups', 'top_ten_historic']

In [37]:
#assign classes/tables to a variable
industry_groups = Base.classes.industry_groups
top_ten = Base.classes.top_ten
historic = Base.classes.top_ten_historic

In [38]:
[column for column in historic.__table__.columns]

[Column('id', INTEGER(), table=<top_ten_historic>, primary_key=True, nullable=False),
 Column('Ticker', VARCHAR(), ForeignKey('top_ten.ticker'), table=<top_ten_historic>),
 Column('Date', DATE(), table=<top_ten_historic>),
 Column('Open', FLOAT(), table=<top_ten_historic>),
 Column('High', FLOAT(), table=<top_ten_historic>),
 Column('Low', FLOAT(), table=<top_ten_historic>),
 Column('Close', FLOAT(), table=<top_ten_historic>),
 Column('Adj Close', FLOAT(), table=<top_ten_historic>),
 Column('Volume', INTEGER(), table=<top_ten_historic>)]

In [39]:
#create a session
session = Session(bind=engine)

In [40]:
#Display the row's columns and data in a dict format
first_row = session.query(historic).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1560afbe0>,
 'Ticker': 'ARB',
 'Date': datetime.date(2018, 11, 5),
 'High': 17.540000915527344,
 'Close': 17.149999618530273,
 'Volume': 1106127,
 'Open': 17.1299991607666,
 'id': 1,
 'Low': 16.3700008392334,
 'Adj Close': None}

In [41]:
# create a new declarative base instance for defining new classes
engine = create_engine('sqlite:///Resources/top_ten_asx.db')
DeclarativeBase = declarative_base()

session = Session(engine)

In [42]:
inspector = inspect(engine)
table_names = inspector.get_table_names()
print(table_names)

['fundamentals', 'industry_groups', 'top_ten', 'top_ten_historic']


In [43]:
inspector.get_columns('top_ten')

[{'name': 'ticker',
  'type': VARCHAR(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'company_name',
  'type': VARCHAR(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'market_cap',
  'type': FLOAT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'industry_id',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]